In [53]:
import numpy as np

dataRaw = np.loadtxt("iris-head-num.txt", delimiter = ",", dtype = "object")

header = dataRaw[0, :]
# print(header)

data = dataRaw[1:,:4]
print(data)

data = np.vstack(data.astype(np.float32))
# print (data)

labels = np.vstack(dataRaw[1:, 4].astype(np.int32))
# print(labels)


[['5.1' '3.5' '1.4' '0.2']
 ['4.9' '3.0' '1.4' '0.2']
 ['4.7' '3.2' '1.3' '0.2']
 ['4.6' '3.1' '1.5' '0.2']
 ['5.0' '3.6' '1.4' '0.2']
 ['5.4' '3.9' '1.7' '0.4']
 ['4.6' '3.4' '1.4' '0.3']
 ['5.0' '3.4' '1.5' '0.2']
 ['4.4' '2.9' '1.4' '0.2']
 ['4.9' '3.1' '1.5' '0.1']
 ['5.4' '3.7' '1.5' '0.2']
 ['4.8' '3.4' '1.6' '0.2']
 ['4.8' '3.0' '1.4' '0.1']
 ['4.3' '3.0' '1.1' '0.1']
 ['5.8' '4.0' '1.2' '0.2']
 ['5.7' '4.4' '1.5' '0.4']
 ['5.4' '3.9' '1.3' '0.4']
 ['5.1' '3.5' '1.4' '0.3']
 ['5.7' '3.8' '1.7' '0.3']
 ['5.1' '3.8' '1.5' '0.3']
 ['5.4' '3.4' '1.7' '0.2']
 ['5.1' '3.7' '1.5' '0.4']
 ['4.6' '3.6' '1.0' '0.2']
 ['5.1' '3.3' '1.7' '0.5']
 ['4.8' '3.4' '1.9' '0.2']
 ['5.0' '3.0' '1.6' '0.2']
 ['5.0' '3.4' '1.6' '0.4']
 ['5.2' '3.5' '1.5' '0.2']
 ['5.2' '3.4' '1.4' '0.2']
 ['4.7' '3.2' '1.6' '0.2']
 ['4.8' '3.1' '1.6' '0.2']
 ['5.4' '3.4' '1.5' '0.4']
 ['5.2' '4.1' '1.5' '0.1']
 ['5.5' '4.2' '1.4' '0.2']
 ['4.9' '3.1' '1.5' '0.1']
 ['5.0' '3.2' '1.2' '0.2']
 ['5.5' '3.5' '1.3' '0.2']
 

In [54]:
# Find unique lables and frequency
labelsUn, labelsCounts = np.unique(labels, return_counts = True)
print(labelsUn)
print(labelsCounts)

[1 2 3]
[50 50 50]


In [55]:
# Find average, max, min and SD of each column per category

nrows, ncols = np.shape(data)
nclasses = len(labelsUn)

average = np.zeros((nclasses, ncols))
maxi = np.zeros((nclasses, ncols))
mini = np.zeros((nclasses, ncols))
sd = np.zeros((nclasses, ncols))

for i in lablesUn:
    indexes = np.reshape(labels == i, nrows)
    average[i - 1, :] = np.mean(data[indexes, :], axis = 0)
    maxi[i - 1, :] = np.max(data[indexes, :], axis = 0)
    mini[i - 1, :] = np.min(data[indexes, :], axis = 0)
    sd[i - 1, :] = np.std(data[indexes, :], axis = 0)

print("average", average)
print("maxi", maxi)
print("mini", mini)
print("sd", sd)

average [[5.00599957 3.41800022 1.46399999 0.24399997]
 [5.93600178 2.76999998 4.26000023 1.32599986]
 [6.58799934 2.97400022 5.55200052 2.02600026]]
maxi [[5.80000019 4.4000001  1.89999998 0.60000002]
 [7.         3.4000001  5.0999999  1.79999995]
 [7.9000001  3.79999995 6.9000001  2.5       ]]
mini [[4.30000019 2.29999995 1.         0.1       ]
 [4.9000001  2.         3.         1.        ]
 [4.9000001  2.20000005 4.5        1.39999998]]
sd [[0.34894693 0.37719488 0.17176732 0.10613199]
 [0.51098341 0.31064451 0.46518815 0.19576517]
 [0.62948871 0.31925544 0.5463478  0.27188972]]


In [56]:
# Find outliers per class and feature based on the formula mean +- 2*sd

outliers2sd = np.zeros((nclasses, ncols))
for i in labelsUn:
    indexes = np.reshape(labels == i, nrows)
    classData = data[indexes, :]
    for j in range(ncols):
        thresholdLow = average[i - 1, j] - 2 * sd[i - 1, j]
        thresholdHigh = average[i - 1, j] + 2 * sd[i - 1, j]
        remain = [x for x in classData[:, j] if (x > thresholdLow)]
        remain = [x for x in classData[:, j] if (x < thresholdHigh)]
        # in percentage
        outliers2sd[i - 1, j] = 100 * (labelsCounts[i - 1] - len(remain)) / labelsCounts[i - 1]

print(outliers2sd)


[[2. 4. 4. 4.]
 [2. 2. 0. 2.]
 [2. 4. 6. 0.]]


In [58]:
# Export to .csv file

decimals = 2
fmt = "%.2f"
formatf = ".csv"
species = np.array(["setosa", "cersicolor", "virginico"])
for i in range(len(labelsUn)):
    temp = np.vstack([average[i, :], mini[i, :], maxi[i, :], sd[i, :], outliers2sd[i, :]]).T
    temp = np.around(temp, decimals)
    temp_str = np.char.mod(fmt, temp)
    rows = np.array(header[: - 1].astype("U"))[:, np.newaxis]
    rowsf = np.hstack((rows, temp_str))
    headerf = [species[i], "mean", "min", "max", "std", "outliers2sd%"]
    np.savetxt(species[i]+formatf, np.vstack((headerf, rowsf)), delimiter = ",", fmt = "%s")